In [19]:

!pip install fitz
!pip install opencv-python
!pip install PyMuPDF


  Obtaining dependency information for fitz from https://files.pythonhosted.org/packages/7e/28/27f27d66eb82f24e6595deb26c0a875e62431878c416e38eac515023abb2/fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata
  Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata (816 bytes)
  Obtaining dependency information for configobj from https://files.pythonhosted.org/packages/d3/bb/d10e531b297dd1d46f6b1fd11d018247af9f2d460037554bb7bb9011c6ac/configobj-5.0.8-py2.py3-none-any.whl.metadata
  Using cached configobj-5.0.8-py2.py3-none-any.whl.metadata (3.4 kB)
  Obtaining dependency information for configparser from https://files.pythonhosted.org/packages/a8/20/f4aab9a42378542295c3be2bbdab353de10eb95396f6d4a5bc7a21b00952/configparser-7.0.0-py3-none-any.whl.metadata
  Using cached configparser-7.0.0-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for httplib2 from https://files.pythonhosted.org/packages/a8/6c/d2fbdaaa5959339d53ba38e94c123e4e84b8fbc4b84beb0e70d7c1608486/httplib2-0

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [286 lines of output]
  Reading version file C:\Users\divya\AppData\Local\Temp\pip-install-75l55no9\traits_87e1d08232ea4998a80e087e29cdf080\traits\version.py
  Package version from version file: ('6.3.2', '6df2ff9bd1d21c74e688aff6f67a19fbbefdd53b')
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\traits
  copying traits\api.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\base_trait_handler.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\constants.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\ctrait.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\editor_factories.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\has_traits.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\interface_

In [20]:
import os
import shutil
# to copy the pdf file into a folder and work on it
def copy_file_to_folder(source_file_path, destination_folder_path):
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder_path, exist_ok=True)

    # Define the destination file path
    destination_file_path = os.path.join(destination_folder_path, os.path.basename(source_file_path))

    # Copy the file
    shutil.copy(source_file_path, destination_file_path)

    print(f'File {source_file_path} copied to {destination_file_path}')


source_file_path = 'grouped_documents (1).pdf'  # Update this with your Python file path
destination_folder_path = '/content/pdfs'  # Update this with the desired folder path

copy_file_to_folder(source_file_path, destination_folder_path)


File grouped_documents (1).pdf copied to /content/pdfs\grouped_documents (1).pdf


In [21]:
#convert the pdfs to images
import fitz
import os

def convert_pdfs_to_images(pdf_folder, output_folder):
    """
    Convert each page of each PDF in the specified folder to PNG images and save them to the output folder.
    """
    # Create the output folder if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    # List all PDF files in the pdf_folder
    pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)


        pdf_document = fitz.open(pdf_path)

        # Convert each page to an image
        for i in range(len(pdf_document)):
            page = pdf_document.load_page(i)
            pix = page.get_pixmap()

            # Generate a unique filename for each page
            image_filename = f"{os.path.splitext(pdf_file)[0]}_page_{i + 1}.png"
            image_path = os.path.join(output_folder, image_filename)

            # Save the image
            pix.save(image_path)


        # Close the PDF file
        pdf_document.close()


pdf_folder = '/content/pdfs'  # Folder containing PDF files
output_folder = '/content/p_images/'  # Folder to save PNG images

convert_pdfs_to_images(pdf_folder, output_folder)

In [22]:
#compute fourier transform of all the images
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image

def fourier_transform_image(image_path):
    """Compute the Fourier Transform of an image."""
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Compute the 2D Fourier Transform
    f_transform = np.fft.fft2(image)
    f_transform_shifted = np.fft.fftshift(f_transform)  # Shift the zero frequency component to the center

    # Compute magnitude spectrum
    magnitude_spectrum = np.abs(f_transform_shifted)

    # Normalize the magnitude spectrum to be in the range [0, 255]
    magnitude_spectrum = 1000*np.log1p(magnitude_spectrum)  # Log scale for better visibility
    magnitude_spectrum = cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX)

    return magnitude_spectrum.astype(np.uint8)

def save_fourier_transform(image_path, output_path):
    """Compute and save the Fourier Transform of an image."""
    magnitude_spectrum = fourier_transform_image(image_path)

    # Save the magnitude spectrum as an image
    output_image = Image.fromarray(magnitude_spectrum)
    output_image.save(output_path)

def process_images_in_folder(input_folder, output_folder):
    """Process all images in the input folder and save their Fourier Transforms."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']
    files = os.listdir(input_folder)
    image_files = [f for f in files if os.path.splitext(f)[1].lower() in image_extensions]

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, f"fourier_{image_file}")
        save_fourier_transform(image_path, output_path)



input_folder = '/content/p_images/'  
output_folder = '/content/fourier'  
process_images_in_folder(input_folder, output_folder)


In [23]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import re

def extract_page_number(filename):
    """
    Extract the page number from a given filename.

    Parameters:
        filename (str): The filename containing the page number.

    Returns:
        int: The extracted page number.
    """
    # Define a regex pattern to extract the page number
    pattern = r'_page_(\d+)\.png$'

    # Search for the pattern in the filename
    match = re.search(pattern, filename)

    if match:
        # Extract and return the page number as an integer
        page_number = int(match.group(1))
        return page_number
    else:
        raise ValueError(f"No page number found in filename: {filename}")

def detect_and_highlight_lines(image_path):
    """
    Detect and highlight lines in the image, and calculate the skew angle.

    Parameters:
        image_path (str): Path to the input image.

    Returns:
        float: The skew angle of the image, adjusted as per the requirement.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 40, 200)

    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
        # Retry with different Canny thresholds if no lines are detected
    if lines is None:
        edges = cv2.Canny(gray, 0, 200)
        lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    # List to store the angles
    angles = []

    # Highlight lines on the original image and calculate angles
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            angle = theta * 180 / np.pi - 90  # Convert theta to degrees and adjust
            angles.append(angle)

            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)



    # Compute the skew angle
    if not angles:
        skew_angle = 0  # Default skew angle to 0 if no lines detected
    else:
        skew_angle = np.median(angles)

    # Adjust the skew angle if positive
    if skew_angle > 0:
        skew_angle = 90 - skew_angle
    # Adjust the skew angle if negative
    elif skew_angle < 0:
        skew_angle = abs(skew_angle) + 90

    return skew_angle

def process_images_in_folder(folder_path):
    """
    Process all images in a folder to compute their skew angles.

    Parameters:
        folder_path (str): Path to the folder containing images.

    Returns:
        list of tuples: Each tuple contains the filename and skew angle.
    """
    # List to store results
    results = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            try:
                skew_angle = detect_and_highlight_lines(image_path)
                page_number = extract_page_number(filename)-1

                results.append((page_number, skew_angle))


            except ValueError as e:
                print(f'{filename}: {e}')

    return results

# Path to your folder containing images
folder_path = '/content/fourier'  # Update this with the path to your folder

# Process all images in the folder
results = process_images_in_folder(folder_path)



In [24]:
#The rotation angles for each page
sorted_list = sorted(results, key=lambda x: x[0])
print(f"Rotation angles for each page: {sorted_list}")

Rotation angles for each page: [(0, 167.00000020742448), (1, 89.99999749552184), (2, 90.99999866171423), (3, 35.49999882483331), (4, 160.00000058181416), (5, 165.99999989500571), (6, 0), (7, 0), (8, 0), (9, 180.0), (10, 180.0), (11, 180.0), (12, 170.9999997495522), (13, 32.99999932444692), (14, 129.00000199589027), (15, 59.000002324692545), (16, 168.99999997848835), (17, 44.999999658377334)]


In [25]:
#delete all the folders and files created during the detection process
import shutil
import os

def delete_folders(folder_paths):
    for folder_path in folder_paths:
        if os.path.exists(folder_path):
            try:
                shutil.rmtree(folder_path)
                print(f"Deleted folder: {folder_path}")
            except Exception as e:
                print(f"Error deleting folder {folder_path}: {e}")
        else:
            print(f"Folder not found: {folder_path}")

# Example list of folders to delete
folders_to_delete = ['/content/p_images', '/content/fourier','/content/pdfs']

# Delete all specified folders
delete_folders(folders_to_delete)


Deleted folder: /content/p_images
Deleted folder: /content/fourier
Deleted folder: /content/pdfs
